# QUBIC monofrequency pipeline

#### Edited by Martín Gamboa, 2022/08/11
#### Edited by Louise Mousset, 2019/11/29

This notebook shows the typical pipeline for data simulation and analysis. There are 2 parts :
* From a given sky map, simulate Time Order Data (TOD) 
* From those TOD, perform Map Making to reconstruct the input sky map

Here we work with only one frequency band.

Style programming guide acording: this link ([qubic wiki](http://qubic.in2p3.fr/wiki/uploads/SimulationsWorkingGroup/20210311OnZoom/collaborative_coding.pdf))

In [ ]:
##Loading modules. 

# General modules
from __future__ import division, print_function
%matplotlib inline
import os
import sys
import time
import datetime
import shutil
from warnings import warn

# Specific science modules
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

# Specific qubic modules
import pysm3
import qubic
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray

from qubic import SpectroImLib as si
from pysm3 import models
from qubic import QubicSkySim as qss

plt.rc('figure', figsize = (13, 10))
plt.rc('font', size = 13)

In [ ]:
# Repository for dictionary
dictfilename = 'explanatory.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

# Set nf_sub to 1 to be in the monofreq case
d['nf_sub'] = 1

# No spectroimaging
d['nf_recon'] = 1

# Center of the patch observed in galactic coordinates
# because the pointing strategy is not in horizontal coordinates (only sweeping pointing it is)
# the center of the FOV has to be in galactic coordinates
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)
d['effective_duration'] = 4
# Adjust some parameters in the dictionary
d['npointings'] = 3000
d['tol'] = 1e-5
d['filter_nu'] = 150e9
# source of noise: photon noise + detector noise. In this case we neglect photon noise
d['photon_noise'] = False
d['noiseless'] = False

print('Central frequency', d['filter_nu'])
print('NEP (detector)', d['detector_nep'])
print('photon noise?', d['photon_noise'])
print('Instrument (TD or FI)', d['config'])
print('Scan duration', d['effective_duration'])

# Input sky
### Some words about maps.

In this case we read a map $X_0$ simulated previously which contains fluctuations in temperature and polarization. Because we are using Healpy module (HEALPix implemented in Python) the number of pixels of each map has to be $$n_{pix} = 12 \times N_{side}^2$$ where $N_{side} = 2^j, \qquad j = 1,2,... $.

In [ ]:
# Make a sky using PYSM
### Pick one of these:
seed = 42
sky_config = {'CMB': 'c1'} ### CMB Only
#sky_config = {'cmb': seed, 'dust':'d1'}   ### CMB + Dust

### Generate the maps at each sub-frequency
Qubic_sky = qss.Qubic_sky(sky_config, d)
input_map = Qubic_sky.get_simple_sky_map()

print('sky shape: ', input_map.shape)

# Look at the input sky maps using Healpy
istokes = 0 # Stokes parameter (I, Q, U)
rr = 9 # Resolution in arcmin

plt.figure(figsize=(13,8))
for istk in range(3):
    plt.subplots_adjust(wspace=0.9)
    hp.mollview(input_map[0, :,istk], cmap = 'jet', sub = (3, 2, 2 * istk + 1), 
                title = 'Mollview {0} Stokes parameter'.format(d['kind'][istk]))
    hp.gnomview(input_map[0, :,istk], cmap = 'jet', sub = (3, 2, 2 * istk + 2), 
                rot = center, reso = rr, 
                title = 'Gnomview {0} Stokes parameter'.format(d['kind'][istk]))

# Time Ordered Data (TOD) simulation

TOD are the signal as a function of time for each bolometer.

$$TOD = AS + n$$

where $A$ is the pointing matrix, $S$ the sky and $n$ the noise.

In [ ]:
# Pointing strategy
pointing = qubic.get_pointing(d)
print('=== Pointing DONE! ===')

# Model of the scene at the time of observation
scene = qubic.QubicScene(d)

# Create a monofrequency Instrument.
qinst = qubic.QubicInstrument(d)

sizeoperators = d['npointings'] * len(qinst) * ((2 * d['synthbeam_kmax']+ 1)**2) * 16/1024**3
sizeoperators = d['nf_sub']*sizeoperators
confirm = input("you will use {:4.3f}Gb, do you want ot continue?".format(sizeoperators))
confirm = bool(confirm)
if (confirm == 1) or (confirm == True): 
    warn("(re)Demodulation confirmed! ")
else:
    sys.exit("Stopped!")
                
# Create an acquisition operator which combines all relevant information
#scene, instrument configuration and pointing strategy. 
acq = qubic.QubicAcquisition(qinst, pointing, scene, d)

# Monofreq TOD making
TOD, maps_convolved = acq.get_observation(input_map[0], noiseless = d['noiseless'])#, convolution = True)

print('TOD shape: ', TOD.shape)
print('maps conv shape: ', maps_convolved.shape)

In [ ]:
# Look at TOD for one TES
print('--------- TOD with shape (#detectors, #pointings) : {} '.format(np.shape(TOD)))

tes = 6
plt.plot(TOD[tes, :], label='TES {}'.format(tes))
plt.xlabel('Pointing index')
plt.ylabel(f'TOD from TES {tes} [W]')
plt.title('FI - {}GHz - Noiseless={} CMB - STD = {:.4} W'.format(int(d['filter_nu']/1e9),
                                                                 d['noiseless'], 
                                                                 np.std(TOD)))

### About memory used 
For end-2-end simulations, the dificulty is the memory requirement. This is why for large number of pointings, you need to parallelize on several machines. The Qubic soft is written in order that parallelization is possible.

During TOD making and map-making, you see this line printed:

    Info shaka: Allocating (2976000,9) elements = 408.69140625 MiB in FSRRotation3dMatrix.__init__.

This is the size of the pointing matrix A. The shape is $(N_{det} x N_{ptgs}, N_{peaks})$ where $N_{det}$ is the number of bolometers (992 for the FI and 248 for the TD), $N_{ptgs}$ the number of pointings and $N_{peaks}$ the number of peaks taken into account in the synthetic beam. 

$N_{peaks}$ is fixed by the synthbeam kmax and the synthbeam fraction both defined in the dictionary.

If the synthbeam fraction is one then $N_{peaks} = (2 \times kmax+1)^2$. Each peak has a given integral, they get smaller as kmax increases. The synthbeam fraction is the fraction of the total integral (over all the peaks) which corresponds to the number of peaks you keep.

**Example**:

    For 9 peaks, with the following repartition: 
        * central peak 50\% of the power in its integral
        * the 4 forming the nearest cross are each 10\%
        * the 4 furthest one each 2.5\%
    Then for $sb_frac = 0.9$, the 4 furthest ones will be eliminated. 

Finally, with $N_{ptgs} = 3000$, $(N_{det}=992$, $kmax=1$ and $sb_frac=1$, the shape of A is (2976000,9).

The **size in MiB** will be: 

In [ ]:
Ndet = len(qinst.detector.index) # Number of detectors
Size = Ndet * d['npointings'] * 9 * 16
print('Size of A:', Size, 'bits')
print('Size of A:', Size/(1024**2), 'MiB')

Note that in case **polyacquisition** (several frequency bands), there is one operator A for each band so the memory required is proportionnal to the number of bands.

Last remark, if you run on several machines **in parallel**, for example at NERSC or at CC-in2p3, the memory is split between the machines. So in the calculation above, you need to divide by the number of machines in order to understand the print:
    
    $ Info moussetDell: Allocating (2976000,9) elements = 408.69140625 MiB in FSRRotation3dMatrix.__init__.


# Coverage map

In [ ]:
# Get coverage map
# i.e. how many times were seen each pixel of the sky (not trivial because of the synthetic beam)
cov_map = acq.get_coverage()

print(cov_map.shape)
hp.mollview(cov_map)

In [ ]:
# Check the pointing and the coverage coincide
theta = pointing.galactic[:, 0]
phi = pointing.galactic[:, 1]
nside = d['nside']

pix = hp.ang2pix(nside, theta, phi, lonlat=True)
ptg_map = np.zeros(12 * nside**2)
ptg_map[pix] = 200
hp.mollview(ptg_map + cov_map)

In [ ]:
hitmap = acq.get_hitmap(d['nside'])
hp.mollview(hitmap)

# Map-making

In [ ]:
# From TOD reconstruct sky maps.
recons_map, nit, error = acq.tod2map(TOD, d, cov = cov_map)

print('The shape of the reconstructed maps is (#pixels, #stokes) :', recons_map.shape)
print('{} iterations were needed to get an error of {}'.format(nit, error))

# Compare input vs output

In [ ]:
# Compare with the convolved maps
diff = recons_map - maps_convolved
print(diff.shape)
print(recons_map.shape)

In [ ]:
# Keeping only the sky region which has been significantly observed
# Pixels not seen enough are replaced by UNSEEN value
maxcov = np.max(cov_map)
unseen = cov_map < maxcov * 0.15

maps_convolved[unseen, :] = hp.UNSEEN
recons_map[unseen, :] = hp.UNSEEN
diff[unseen, :] = hp.UNSEEN

In [ ]:
rr = 14 # Resolution in arcmin
stokes = ['I', 'Q', 'U']
plt.figure(figsize=(15, 15))
for istokes in range(3):
    if istokes == 0:
        min = -200
        max = 200
    else:
        min = -8
        max = 8
    hp.gnomview(maps_convolved[:, istokes], cmap = 'jet', 
                rot = center, sub = (3,3,3*istokes+1), reso = rr,
                title = 'Input ' + stokes[istokes], unit = '$\mu K_{CMB}$', 
                format = '%g',  min = min, max = max)
    hp.gnomview(recons_map[:, istokes], cmap = 'jet',
                rot = center, sub = (3,3,3*istokes+2), reso = rr,
                title = 'Output ' + stokes[istokes], unit = '$\mu K_{CMB}$', 
                min = min, max = max)
    hp.gnomview(diff[:, istokes], cmap = 'jet', 
                rot = center, sub = (3,3,3*istokes+3), reso = rr,
                title = 'Difference ' + stokes[istokes], unit = '$\mu K_{CMB}$', 
                min = -2, max = 2)
hp.graticule(dpar = 5, dmer = 5, verbose = False, alpha = 0.5)

In [ ]:
seenpix = cov_map > maxcov * 0.3
print(np.std(diff[seenpix, 2]))

map_convcenter = np.copy(maps_convolved)
map_reconcenter = np.copy(recons_map)
diffcenter = np.copy(diff)

map_convcenter[~seenpix, :] = hp.UNSEEN
map_reconcenter[~seenpix, :] = hp.UNSEEN
diffcenter[~seenpix, :] = hp.UNSEEN
for istokes in range(3):
    if istokes == 0:
        min = None
        max = None
    else:
        min = None
        max = None
    hp.gnomview(map_convcenter[:, istokes], cmap='jet', rot=center, sub=(3,3,3*istokes+1), reso=rr,
                title='Input ' + stokes[istokes], notext=True, min=min, max=max)
    hp.gnomview(map_reconcenter[:, istokes], cmap='jet',rot=center, sub=(3,3,3*istokes+2), reso=rr,
                title='Output ' + stokes[istokes], notext=True, min=min, max=max)
    hp.gnomview(diffcenter[:, istokes], cmap='jet',rot=center, sub=(3,3,3*istokes+3), reso=rr,
                title='Difference ' + stokes[istokes], notext=True, min=None, max=None)